# MODELOS

#### Importar datos y explorar 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#leer archivo limpio 

df = pd.read_csv('auto-mpg.data')
#explorar df
df.head()

#### Definir variable dependiente del modelo

In [ ]:
#Modelo de regresion--> variable y = precio . LISTA de nombre
y_reg= []

#Modelo de clasificacion --> variable y = rating . LISTA de nombre
y_clas= []

## MODELO DE REGRESIÓN

#### Exploracion de variables X importante

In [ ]:
# Histogramas de cada variable y dispersion entre variables 
import seaborn as sns
sns.pairplot(df,)

#Ver correlacion entre datos 
corr = df.corr()[y_reg].sort_values(ascending=False)
sb.heatmap(corr, cmap="Blues", annot=True)

# Ver dispersion y tendencia entre variables x y y
#Definir variables x totales. LISTA
x_completo_reg=[]
sns.pairplot(df,x_vars= x_completo_reg, y_vars=y_reg, height=7, aspect=0.7, kind='reg')

In [ ]:
#ANOVA para ver relevancia estadistica de variables 

import statsmodels.api as sm
from statsmodels.formula.api import ols

modelo = ols("price ~ C(neighbourhood_cleansed)", data=df).fit()
sm.stats.anova_lm(modelo, typ=2) 


#### Seleccionar variables x iniciales

In [ ]:

df.key()
#nombrar lista de variables x que se consideran relevantes
x_relevante_reg=[]

x
#crear nuevo dataframe con variables relevantes
datos=df.[[y,x_relevante_reg]].copy()

#Dividir variables x numericas
x_num=[]
#Dividir variables x categoricas
x_cat=[]

#### Dividir datos 

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Listas de tus variables
columnas_numericas = ["accommodates", "bedrooms", "bathrooms", "minimum_nights"]
columnas_categoricas = ["neighbourhood_cleansed", "room_type"]

# Definir el preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), columnas_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), columnas_categoricas),
    ]
)

#### Codificar variables dummies


In [ ]:
data = pd.get_dummies(df, columns=['Origin'], prefix='', prefix_sep='', dtype=int)

### Modelos de regresion- precios

### Modelos de clasificacion- alquilar o no

In [ ]:
print ()